This notebook demonstrates how we used Wikidata to derive direct mappings between SSH-LCSH vocabulary concepts and Getty AAT concepts

In [2]:
"""
=============================================================================
Module    : get-SSH-LCSH-AAT-mappings.ipynb
Classes   : 
Project   : ATRIUM
Creator   : Ceri Binding, University of South Wales / Prifysgol de Cymru
Contact   : ceri.binding@southwales.ac.uk
Summary   : Python notebook demonstrating use of Wikidata to derive vocab mappings
Imports   : os, requests, pyoxigraph, pathlib, rdflib, SPARQLWrapper, datetime
License   : https://github.com/cbinding/ATRIUM-data/blob/main/LICENSE.txt
=============================================================================
History
18/042/2024 CFB Initially created script
=============================================================================
"""
# load required dependencies
%pip install --upgrade pip
%pip install -r "./requirements.txt"

# import required library modules
import os
import requests
from pyoxigraph import *
from pathlib import Path
from rdflib import ConjunctiveGraph, URIRef, Literal
from SPARQLWrapper import SPARQLWrapper, RDFXML
from datetime import datetime as DT

# function to run SPARQL query against specified external endpoint
# returnFormat supports "xml", "n3", "turtle", "nt", "pretty-xml", "trix", "trig", "nquads", "json-ld" and "hext"
def querySparqlEndpoint(endpointURI: str="", sparqlQuery: str="", returnFormat: str="nt"):
  sparql = SPARQLWrapper(endpointURI)
  sparql.setQuery(sparqlQuery)
  sparql.setMethod("POST")
  sparql.setReturnFormat(RDFXML)
  results = sparql.queryAndConvert()
  return results.serialize(format=returnFormat) 

def timestamp():
  return DT.now().strftime('%Y-%m-%dT%H:%M:%SZ')  

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


The TRIPLE Vocabulary ([doi:10.12681/semantics.gr/SSH-LCSH](https://doi.org/10.12681/semantics.gr/SSH-LCSH)) is an SSH multilingual vocabulary based on a subset of Library of Congress Subject Headings (LCSH). The site reports there are currently 3375 semantic resources (concepts).<br>First get the SSH-LCSH vocabulary data and save to a local file (here using NTriples format) 

In [6]:
url = "https://www.semantics.gr/authorities/vocabularies/SSH-LCSH/n-triples"
response = requests.get(url, timeout=30)
with open("./data/SSH-LCSH.nt", "w") as file:
    file.write(response.text)

print(f"[last run {timestamp()}]")

[last run 2024-04-24T08:43:32Z]


Query the [Getty vocabularies SPARQL endpoint](https://vocab.getty.edu/sparql) to retrieve Art &amp; Architecture Thesaurus (AAT) concept URIs with associated preferred labels (English). These labels are used for review of the mappings produced later. (Note results reported here are min 2 triples per concept)

In [18]:
# Getty AAT SPARQL query to get prefLabels (English) for each Concept (saved as NTriples output)
query = """
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX gvp: <http://vocab.getty.edu/ontology#>
CONSTRUCT { 
  ?uri skos:prefLabel ?lbl . 
  ?uri a gvp:Concept .
}
#SELECT ?uri (STR(?lbl) AS ?label) 
WHERE {
 ?uri a gvp:Concept; skos:inScheme aat:; 
	gvp:prefLabelGVP [gvp:term ?lbl] .
 FILTER(langMatches(lang(?lbl), "en"))
}
"""
results = querySparqlEndpoint(endpointURI="https://vocab.getty.edu/sparql", sparqlQuery=query)
with open("./data/AAT-prefLabels.nt", "w") as file: 
  file.write(results)
  
counter = len(results.split("\n"))
print(f"{counter} results [last run {timestamp()}]")

116343 results [last run 2024-04-23T14:40:14Z]


Query the Wikidata SPARQL endpoint (https://query.wikidata.org/) to retrieve &lt;LCSH URI&gt; skos:closeMatch &lt;AAT URI&gt; mappings and save them to an NTriples format file. The mappings are generated from Wikidata records having both LCSH and AAT identifier properties. Note: Wikidata LCSH URIs retrieved using the wdtn:P244 property are prefixed 'http://id.loc.gov/authorities/names/' - but this appears to be a mistake when referring to subject terms (e.g. http://www.wikidata.org/entity/Q167555 references http://id.loc.gov/authorities/names/sh85095140 but this should be http://id.loc.gov/authorities/subjects/sh85095140). In order to match the LCSH URIs as referenced (correctly) in SSH-LCSH vocabulary, we are concatenating the correct prefix to the LCSH concept identifier, instead of using the URI as retrieved from Wikidata.

In [7]:
# Wikidata SPARQL query to generate LCSH-closeMatch-AAT mappings (saved as NTriples output)
query = """
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX gvp: <http://vocab.getty.edu/ontology#>
CONSTRUCT { ?lcsh_uri skos:closeMatch ?aat_uri }
WHERE { 
  ?wd_uri wdt:P244 ?lcsh_id; wdtn:P1014 ?aat_uri .
  BIND(IRI(CONCAT('http://id.loc.gov/authorities/subjects/', ?lcsh_id)) AS ?lcsh_uri) .
}
"""
results = querySparqlEndpoint(endpointURI="https://query.wikidata.org/sparql", sparqlQuery=query)
with open("./data/LCSH-closeMatch-AAT.nt", "w") as file: 
  file.write(results)
  
counter = len(results.split("\n"))
print(f"{counter} results [last run {timestamp()}]")

7946 results [last run 2024-04-24T08:43:55Z]


Combine the SSH-LCSH vocabulary, the AAT data and the LCSH-closeMatch-AAT mappings to an rdflib ConjunctiveGraph instance. 

In [3]:

graph = ConjunctiveGraph(store="Oxigraph") # speed advantage over plain rdflib.Graph
# persisting to disk. Not strictly necessary here though
# if not os.path.isdir(os.path.join(os.getcwd(), "graph_dir")): graph.open("graph_dir")
# import RDF files
graph.parse("./data/SSH-LCSH.nt", publicID="http://example2.com")
graph.parse("./data/AAT-prefLabels.nt", publicID="http://example3.com")
graph.parse("./data/LCSH-closeMatch-AAT.nt", publicID="http://example4.com")

<Graph identifier=N7600fcd01cc34b77b740a85830144cae (<class 'rdflib.graph.Graph'>)>

Use the LCSH-closeMatch-AAT mappings to produce SSH-LCSH-closeMatch-AAT mappings. Save results to an NTriples RDF file, then import the newly produced mappings back into the graph triple store.

In [4]:

query = """
  PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
  PREFIX gvp: <http://vocab.getty.edu/ontology#>
  CONSTRUCT { ?ssh_uri skos:closeMatch ?aat_uri }
  WHERE {
    ?ssh_uri a skos:Concept; skos:exactMatch ?lcsh_uri .
    ?lcsh_uri skos:closeMatch ?aat_uri .
    ?aat_uri a gvp:Concept .
}
"""
results = graph.query(query)

# save the resultant constructed mappings to an NTriples RDF format text file
results.serialize(destination="./data/SSH-LCSH-closeMatch-AAT.nt", format="nt")
# add these mappings back into the existing graph
graph.parse("./data/SSH-LCSH-closeMatch-AAT.nt", publicID="http://example5.com")

# function to check how many SSH-LCSH concepts remain unmapped
def countUnmappedConcepts():
    query = """
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        PREFIX gvp: <http://vocab.getty.edu/ontology#>
        SELECT (COUNT(DISTINCT ?ssh_uri) AS ?counter) WHERE {
        ?ssh_uri a skos:Concept .
        MINUS { ?ssh_uri ?property [a gvp:Concept] }
        }
    """
    output = graph.query(query)
    # we only want the count (maybe a better way to do this?)
    for result in output:
        return result[0]
        break    

unmapped = countUnmappedConcepts()
print(f"{len(results)} results. {unmapped} unmapped concepts [last run {timestamp()}]")

393 results. 2982 unmapped concepts [last run 2024-04-24T13:02:51Z]


For SSH-LCSH concepts not yet mapped, if the immediate parent concept has a skos:closeMatch mapping to an AAT concept, create a skos:broadMatch mapping to that concept

In [5]:
# create skos:broadMatch relationships for concepts
# not yet mapped, using vocabulary broader relationship
query = """
  PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
  PREFIX gvp: <http://vocab.getty.edu/ontology#>
  CONSTRUCT { ?ssh_uri skos:broadMatch ?aat_uri }
  WHERE {
    ?ssh_uri a skos:Concept; skos:broader ?broader_uri .
    ?broader_uri a skos:Concept; skos:closeMatch ?aat_uri .
    ?aat_uri a gvp:Concept .
    MINUS { ?ssh_uri ?property [a gvp:Concept] }
  }
"""
results = graph.query(query)

# save the resultant constructed mappings to an NTriples RDF format text file
results.serialize(destination="./data/SSH-LCSH-broadMatch-AAT.nt", format="nt")
# add the resultant mappings back into the existing graph
graph.parse("./data/SSH-LCSH-broadMatch-AAT.nt")

unmapped = countUnmappedConcepts()
print(f"{len(results)} results. {unmapped} unmapped concepts [last run {timestamp()}]")

3347 results. 665 unmapped concepts [last run 2024-04-24T13:02:56Z]


For SSH-LCSH concepts not yet mapped, if the grandparent concept has a skos:closeMatch mapping to an AAT concept, create a skos:broadMatch mapping to that concept

In [6]:
# create skos:broadMatch relationships  for concepts
# not yet mapped, using TWO steps of broader relationship 
query = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX gvp: <http://vocab.getty.edu/ontology#>
    CONSTRUCT { ?ssh_uri skos:broadMatch ?aat_uri }
    WHERE {
    ?ssh_uri a skos:Concept; skos:broader/skos:broader ?broader_uri .
    ?broader_uri a skos:Concept; skos:closeMatch ?aat_uri .
    ?aat_uri a gvp:Concept .
    MINUS { ?ssh_uri ?property [a gvp:Concept] }
    }
"""
results = graph.query(query)

# save the resultant constructed mappings to an NTriples RDF format text file
results.serialize(destination="./data/SSH-LCSH-broadbroadMatch-AAT.nt", format="nt")
# add the resultant mappings to the existing graph
graph.parse("./data/SSH-LCSH-broadbroadMatch-AAT.nt")

unmapped = countUnmappedConcepts()
print(f"{len(results)} results. {unmapped} unmapped concepts [last run {timestamp()}]")

367 results. 398 unmapped concepts [last run 2024-04-24T13:02:59Z]


Create a CSV listing of the mappings created, for manual review

In [7]:
# list the mappings created (for review)
query = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX gvp: <http://vocab.getty.edu/ontology#>
    SELECT DISTINCT ?ssh_uri (str(?ssh_lbl) AS ?ssh_label) ?rel ?aat_uri (str(?aat_lbl) AS ?aat_label)    
    WHERE {
        ?ssh_uri skos:closeMatch|skos:broadMatch ?aat_uri . 
        ?ssh_uri a skos:Concept .
        ?aat_uri a gvp:Concept .         
        ?ssh_uri ?rel ?aat_uri .        
        OPTIONAL { 
            ?ssh_uri skos:prefLabel ?ssh_lbl . 
            FILTER(langMatches(lang(?ssh_lbl), "en")) 
        }
        OPTIONAL { 
            ?aat_uri skos:prefLabel ?aat_lbl . 
            FILTER(langMatches(lang(?aat_lbl), "en")) 
        }
    }
"""
mappings = graph.query(query)
mappings.serialize(destination="./data/SSH-LCSH-matched-AAT.csv", format="csv")

print(f"{len(mappings)} mappings [last run {timestamp()}]")

4139 mappings [last run 2024-04-24T13:03:02Z]


Display some of the mappings produced here, for review. Note SSH-LCSH records showing a blank label are just those where there is no English preferred label present in the vocabulary, the mapping still works regardless of this

In [8]:
# display some of the mappings produced for review
import pandas as pd
from IPython.display import display, HTML
df = pd.DataFrame(mappings)
display(HTML(df.fillna("").to_html(index=False, header=False, max_rows=25)))

http://semantics.gr/authorities/SSH-LCSH/sh85028889,,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300055912,commedia dell'arte
http://semantics.gr/authorities/SSH-LCSH/sh85081863,Mass media,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300055812,mass media
http://semantics.gr/authorities/SSH-LCSH/sh85067272,Interior decoration,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300161596,interior decoration
http://semantics.gr/authorities/SSH-LCSH/sh85066150,Information science,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300054574,information science
http://semantics.gr/authorities/SSH-LCSH/sh85076723,Library science,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300054576,library science
http://semantics.gr/authorities/SSH-LCSH/sh85108411,Psychoanalysis,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300054450,psychoanalysis
http://semantics.gr/authorities/SSH-LCSH/sh85148092,Word (Linguistics),http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300250895,words
http://semantics.gr/authorities/SSH-LCSH/sh85036316,Decorative arts,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300054168,decorative arts
http://semantics.gr/authorities/SSH-LCSH/sh85119950,Semiotics,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300054254,semiotics
http://semantics.gr/authorities/SSH-LCSH/sh85015738,Books,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300028051,books
http://semantics.gr/authorities/SSH-LCSH/sh85006611,Architecture,http://www.w3.org/2004/02/skos/core#closeMatch,http://vocab.getty.edu/aat/300263552,architecture


Produce a CSV listing of the remaining SSH-LCSH concepts not yet mapped, for manual review

In [9]:
# list the remaining records not mapped
query = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX gvp: <http://vocab.getty.edu/ontology#>
    SELECT DISTINCT ?ssh_uri (str(?lbl) AS ?label) WHERE {
    ?ssh_uri a skos:Concept .
    OPTIONAL { 
        ?ssh_uri skos:prefLabel ?lbl . 
        FILTER(langMatches(lang(?lbl), "en")) 
    }
    MINUS { ?ssh_uri ?property [a gvp:Concept] }
    }
"""
unmapped = graph.query(query)
unmapped.serialize(destination="./data/SSH-LCSH-nomatch-AAT.csv", format="csv")

print(f"{len(unmapped)} unmapped [last run {timestamp()}]")

398 unmapped [last run 2024-04-24T13:03:12Z]


Display some of the unmapped SSH-LCSH concepts here, for review.

In [10]:
# display the unmapped records here for review
import pandas as pd
from IPython.display import display, HTML
df = pd.DataFrame(unmapped)
display(HTML(df.fillna("").to_html(index=False, header=False, max_rows=25)))

http://semantics.gr/authorities/SSH-LCSH/sh85116920,Salvadoran literature
http://semantics.gr/authorities/SSH-LCSH/sh85072864,Komi-Permyak literature
http://semantics.gr/authorities/SSH-LCSH/sh85057146,"Greek essays, Modern"
http://semantics.gr/authorities/SSH-LCSH/sh85048416,Finnish literature
http://semantics.gr/authorities/SSH-LCSH/sh85054380,German literature
http://semantics.gr/authorities/SSH-LCSH/sh88005294,"Rwanda, Literatures"
http://semantics.gr/authorities/SSH-LCSH/sh85086828,Mongolian literature
http://semantics.gr/authorities/SSH-LCSH/sh85119451,Sects
http://semantics.gr/authorities/SSH-LCSH/sh85028528,Colombian literature
http://semantics.gr/authorities/SSH-LCSH/sh85073564,
http://semantics.gr/authorities/SSH-LCSH/sh85126289,Spanish poetry


Create a listing of all generated mappings in [TriG](https://www.w3.org/TR/trig/) RDF format

In [14]:
from rdflib.namespace import XSD

# remove the named graph of mappings if it already exists
graphURI = "http://mappings-SSH-LCSH-AAT/"
mappings = graph.get_context(URIRef(graphURI))
graph.remove_context(mappings)

# get the generated mappings as RDF NTriples
query = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX gvp: <http://vocab.getty.edu/ontology#>
    CONSTRUCT { ?ssh_uri ?rel ?aat_uri }
    WHERE {  
    ?ssh_uri skos:closeMatch|skos:broadMatch ?aat_uri .
    ?ssh_uri a skos:Concept . 
    ?aat_uri a gvp:Concept .
    ?ssh_uri ?rel ?aat_uri . 
    } 
"""
results = graph.query(query)
# add named graph URI to each result for subsequent quads output
results = list(map(lambda t: t + (graphURI,), results))
# add the mappings (now quads) to the triple store
graph.addN(results)

# add some metadata to the named graph of mappings
graph.addN([
    (
        URIRef(graphURI),
        URIRef("http://purl.org/dc/elements/1.1/identifier"),
        Literal(graphURI),
        URIRef(graphURI),
    ),    
    (
        URIRef(graphURI),
        URIRef("http://purl.org/dc/elements/1.1/date"),
        Literal(timestamp(), datatype=XSD.date),
        URIRef(graphURI),
    ),
    (
        URIRef(graphURI),
        URIRef("http://purl.org/dc/elements/1.1/source"),
        Literal("https://query.wikidata.org/"),
        URIRef(graphURI),
    ),    
    (
        URIRef(graphURI),
        URIRef("http://purl.org/dc/elements/1.1/title"),
        Literal("SSH-LCSH-AAT mappings"),
        URIRef(graphURI),
    ),
    (
        URIRef(graphURI),
        URIRef("http://purl.org/dc/elements/1.1/description"),
        Literal("SSH-LCSH-AAT mappings generated from Wikidata"),
        URIRef(graphURI),
    ),
    (
        URIRef(graphURI),
        URIRef("http://purl.org/dc/elements/1.1/format"),
        Literal("application/trig"),
        URIRef(graphURI),
    )
])


# export the entire named graph of mappings to TRIG format RDF file
mappings = graph.get_context(URIRef(graphURI))
mappings.serialize(destination="./data/SSH-LCSH-matched-AAT.trig", format="trig")

print(f"{len(results)} results [last run {timestamp()}]")

4137 results [last run 2024-04-24T13:08:36Z]
